<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js017_mixMMD2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### html + css + js in the code cell

In [ ]:
#@title js017_mixMMD2D

%%html

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js017_mixMMD2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js017_mixMMD2D
//    Copyright(C) 2017 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.05.07 created, last updated on 2018.11.15
//    ver 0.0.1  2019.01.13 v1, last updated on 2021.05.06
//    ver 0.0.2  2021.10.29 v2, last updated on 2021.10.29
//    ver 0.0.3  2023.03.05 v3, last updated on 2023.08.16
//
// --------------------  molecular dynamics 2D
//
//    method: velocity Verlet Algorithm
//      (1) vi = vi + (Fi/mi)*(0.5dt)
//      (2) ri = ri + vi*dt
//      (3) calculation Fi <- {r1,r2,...,rn} Fi=sum(Fij,j=1 to n),Fij=F(ri-rj)
//      (4) vi = vi + (Fi/mi)*(0.5dt)
//      goto (1)
//
//   potential: Morse V(r) = D*((1-EXP(-A*(r-r0)))^2-1)
//                        = D*(EXP(-2*A*(r-r0))-2*EXP(-A*(r-r0)))
//     (D:dissociation energy, r0:bond length, A:width parameter { A=SQR(k/(2*D)) }
//             force F(r) = -dV(r)/dr
//                        = 2*D*A*y*(y-1), y=EXP(-A*(r-r0))
//
//    for fast calculation
//      ignore F(r) r>rCutoff
//      force F(r) <- force table + linear interpolation (see setForceTable() and cutoff(r))
//      register near particles reg[][] (see registerNearParticles()), 'near' means r<rCutoff+20*2000*dt
//        reg[][] use 20 times, assuming particle max speed < 2000m/s
//      force calculation: sum up force(r) (r<rCutoff)
//
// --------------------
*/

const mixMMD2D = (function() { // ====================  mixMMD2D Module  ====================

	const g_AMU = 1.66053904e-27;				// (kg) atomic mass unit
	const g_kB = 1.380649e-23;					// (J/K) Boltzmann's constant
	const g_EE = 1.602176634e-19;				// (C) electron charge, energy : 1(eV) = EE(J)
	const g_nMax = 2000;						// array max

	let g_molecKind1 = 3;						// 3:Fe
	let g_molecKind2 = 5;						// 5:Al
	let g_nParticles = 100;						// number of particles
	let g_sysTime = 0.0;						// (s) system time
	let g_timeStep =  5.0*1.0e-15;				// (s) time step
	let g_xMax = 6.0E-9;						// (m) x-Box size
	let g_yMax = 6.0E-9;						// (m) y-Box size
	let g_kineticEnergy = 0.0;					// (J) total kinetic energy
	let g_potentialEnergy = 0.0;				// (J) total potential energy
	let g_rCutoff = 1.0e-9;						// force cutoff length
	let g_hh = 1.0e-12;							// forceTable r-division

	const g_xx = dim1( g_nMax );				// (m) x-component of i-th particle position
	const g_yy = dim1( g_nMax );				// (m) y-component of i-th particle position
	const g_vx = dim1( g_nMax );				// (m/s) x-component of i-th particle velocity
	const g_vy = dim1( g_nMax );				// (m/s) y-component of i-th particle velocity
	const g_ffx = dim1( g_nMax );				// (N) x-component of total force applied i-th particle
	const g_ffy = dim1( g_nMax );				// (N) y-component of total force applied i-th particle
	const g_mas = dim1( g_nMax );				// (kg) mass of i-th particle
	const g_kind = dimInt1( g_nMax );			// kind of i-th particle
	const g_potentialTable = dim3( 21, 21, 1002 ); // potential table(ki,kj) [V[0], V[hh], V[2hh],..., V[rCutoff]]
	const g_forceTable = dim3( 21, 21, 1002 );	// force table(ki,kj) [F[0], F[hh], F[2hh],..., F[rCutoff]]
	const g_reg = dimInt2( g_nMax, 100 );		// reg[i][0]:total number of particles near i-th particle
												// reg[i][j]:particle number near i-th particle, (j>0)

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dimInt1( n ) {
		return new Int32Array( n );
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}

	function dim3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = new Float64Array( nk );
			}
		}
		return a;
	}


	// --------------------  Morse potential data  --------------------

	//   0 mass(AU),      1 dd(eV),     2 aa(1/m), 3 r0(m),   4 col,     5 molecStr
	const g_Morse = [
		[ 183.85 *g_AMU, 0.9906*g_EE, 1.4116e10, 3.032e-10, "#6622ff", "W"  ], //  0
		[  95.94 *g_AMU, 0.8032*g_EE, 1.5079e10, 2.976e-10, "#4444ff", "Mo" ], //  1
		[  51.996*g_AMU, 0.4414*g_EE, 1.5721e10, 2.754e-10, "#0088ff", "Cr" ], //  2
		[  55.847*g_AMU, 0.4174*g_EE, 1.3885e10, 2.845e-10, "#00ffff", "Fe" ], //  3
		[  58.71 *g_AMU, 0.4205*g_EE, 1.4199e10, 2.780e-10, "#00ff88", "Ni" ], //  4
		[  26.98 *g_AMU, 0.2703*g_EE, 1.1646e10, 3.253e-10, "#00ff00", "Al" ], //  5
		[ 207.19 *g_AMU, 0.2348*g_EE, 1.1836e10, 3.733e-10, "#88ff00", "Pb" ], //  6
		[  63.54 *g_AMU, 0.3429*g_EE, 1.3588e10, 2.866e-10, "#cccc00", "Cu" ], //  7
		[ 107.87 *g_AMU, 0.3323*g_EE, 1.3690e10, 3.115e-10, "#bbbb66", "Ag" ], //  8
		[  40.08 *g_AMU, 0.1623*g_EE, 0.8054e10, 4.569e-10, "#ddaa00", "Ca" ], //  9
		[  87.62 *g_AMU, 0.1513*g_EE, 0.7878e10, 4.988e-10, "#ff8800", "Sr" ], // 10
		[ 137.34 *g_AMU, 0.1416*g_EE, 0.6570e10, 5.373e-10, "#ff4400", "Ba" ], // 11
		[  22.99 *g_AMU, 0.0633*g_EE, 0.5900e10, 5.336e-10, "#ff0000", "Na" ], // 12
		[  39.102*g_AMU, 0.0542*g_EE, 0.4977e10, 6.369e-10, "#ff0044", "K"  ], // 13
		[  85.47 *g_AMU, 0.0464*g_EE, 0.4298e10, 7.207e-10, "#ff0088", "Rb" ], // 14
		[ 132.905*g_AMU, 0.0449*g_EE, 0.4157e10, 7.557e-10, "#ff00cc", "Cs" ], // 15
		[  20.183*g_AMU, 0.0031*g_EE, 1.6500e10, 3.076e-10, "#ff00ff", "Ne" ], // 16
		[  39.948*g_AMU, 0.0104*g_EE, 1.3400e10, 3.816e-10, "#cc44ff", "Ar" ], // 17
		[  83.80 *g_AMU, 0.0141*g_EE, 1.2500e10, 4.097e-10, "#8844ff", "Kr" ], // 18
		[ 131.30 *g_AMU, 0.0200*g_EE, 1.2400e10, 4.467e-10, "#4488ff", "Xe" ], // 19
		[ 200.59 *g_AMU, 0.0734*g_EE, 1.4900e10, 3.255e-10, "#00ccff", "Hg" ] ]; // 20

	const g_rCollision = [];
	const g_rBond = [];
	const g_massOf = [];
	const g_strOf = [];
	const g_colorOf = [];		// g_colorOf[kind] : color of kind

	(function() {
		const n=g_Morse.length;

		for (let i=0; i<n; i++) {
			g_rCollision[i] = 0.5*g_Morse[i][3]/1.12246;
			g_rBond[i] = 0.5*g_Morse[i][3];
			g_massOf[i] = g_Morse[i][0];
			g_strOf[i] = g_Morse[i][5];
			g_colorOf[i] = g_Morse[i][4];
		}
	}());

	function colorStrOf(kind) {
		return "<span style='color:"+g_colorOf[kind]+"'>"+g_strOf[kind]+"</span>";
	}


	//--- set forceTable[kindi][kindj][ir] and potentialTable[kindi][kindj][ir]

	function setForceTable() {
		const hh=g_hh;

		for (let i=0; i<21; i++) {
			for (let j=0; j<21; j++) {
				const dd = Math.sqrt(g_Morse[i][1]*g_Morse[j][1]);
				const aa = 0.5*(g_Morse[i][2]+g_Morse[j][2]);
				const r0 = 0.5*(g_Morse[i][3]+g_Morse[j][3]);
				for (let ir=1; ir<=1001; ir++) {
					const r = ir*hh;
					const y = Math.exp(-aa*(r-r0));
					g_potentialTable[i][j][ir] = cutoff(r)*dd*y*(y-2.0); // V(r) = D*((1-y)^2-1) = D*y*(y-2)
					//g_forceTable[i][j][ir] = cutoff(r)*2.0*dd*aa*y*(y-1);
				}
				g_potentialTable[i][j][0] = g_potentialTable[i][j][1] + g_potentialTable[i][j][2];
				for (let ir=1; ir<=1000; ir++) {
					g_forceTable[i][j][ir] = -(g_potentialTable[i][j][ir+1] - g_potentialTable[i][j][ir-1])/(2.0*hh);
				}
				g_forceTable[i][j][1001] = -(0.0 - g_potentialTable[i][j][1000])/(2.0*hh);
				g_forceTable[i][j][0] = g_forceTable[i][j][1];
			}
		}
	}

	function cutoff(r) {
		let ret;

		if (r>0 && r<0.8*g_rCutoff) {
			ret = 1.0;
		} else if (r>=0.8*g_rCutoff && r<g_rCutoff ) {
			ret = 0.5+0.5*Math.cos(Math.PI*(r-0.8*g_rCutoff)/(0.2*g_rCutoff));
		} else {
			ret = 0.0;
		}
		return ret;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( theme, boxSizeInNM, contTemp ) {
		setForceTable();
		g_sysTime = 0.0;
		g_xMax = boxSizeInNM*1.0e-9;
		g_yMax = boxSizeInNM*1.0e-9;

		let knd1, knd2;
		if (theme==0) { // 0:Fe-Al
			knd1 = 3; knd2 = 5;
		} else if (theme==1) { // 1:Ni-Pb
			knd1 = 4; knd2 = 6;
		} else if (theme==2) { // 2:Ni-Cr
			knd1 = 4; knd2 = 2;
		} else { // Fe-Al
			knd1 = 3; knd2 = 5;
		}
		setCondition(knd1,knd2,boxSizeInNM,contTemp);
	}

	function setCondition(knd1,knd2,boxSizeInNM,contTemp) {
		g_molecKind1 = knd1;
		g_molecKind2 = knd2;
		g_rCutoff = Math.min(Math.max(3.0*g_Morse[g_molecKind1][3],3.0*g_Morse[g_molecKind2][3]),1.0e-9);

		const s = 0.1*g_xMax;
		const xtalSize = 0.35*g_xMax;
		const ss = 0.55*g_xMax;
		let ii = 0;
		ii = setCrystalBlock(ii, knd1, s, s, xtalSize, xtalSize, Math.PI/4);
		ii = setCrystalBlock(ii, knd2, s, ss, xtalSize, xtalSize, 0);
		ii = setCrystalBlock(ii, knd2, ss, s, xtalSize, xtalSize, 0)
		g_nParticles = setCrystalBlock(ii, knd1, ss, ss, xtalSize, xtalSize, 0);
		ajustVelocity(contTemp);
	}

	function setCrystalBlock(ii, knd, x0, y0, xLen, yLen, theta) {
		let iip = ii;
		const a = 0.98*2.0*g_rBond[knd];
		const b = 0.866025*a;
		let leng = xLen;
		if (leng<yLen) leng = yLen;
		leng = 1.5*leng;
		const nx = Math.floor(leng/b) + 1;
		const ny = Math.floor(leng/a) + 1;
		const sth = Math.sin(theta);
		const cth = Math.cos(theta);
		for (let i=0; i<nx; i++) {
			const x = b*i - leng/2.0;
			for (let j=0; j<ny; j++) {
				let y = a*j - leng/2.0;
				if ((i%2)==1) y = y + 0.5*a;
				const xp = x0 + xLen/2.0 + cth*x - sth*y;
				const yp = y0 + yLen/2.0 + sth*x + cth*y;
				if (xp>=x0 && xp<=x0+xLen && yp>=y0 && yp<=y0+yLen) {
					setParticle(iip, knd, xp, yp);
					iip = iip + 1;
				}
			}
		}
		return iip;
	}

	function setParticle(i, knd, x, y) {
		g_xx[i] = x;
		g_yy[i] = y;
		g_vx[i] = 200.0*normalRandom3();
		g_vy[i] = 200.0*normalRandom3();
		g_ffx[i] = 0.0;
		g_ffy[i] = 0.0;
		g_kind[i] = knd;
		g_mas[i] = g_massOf[knd];
	}

	// normal distributed random number: -3.0 <= normalRandom3() < 3.0
	function normalRandom3() {
		return (Math.random()+Math.random()+Math.random()+Math.random()+Math.random()+Math.random()-3.0);
	}


	// --------------------  time evolution  --------------------

	function timeEvolution(tempMode,contTemp) {
		if (tempMode==1) ajustVelocity(contTemp);
		registerNearParticles();
		for (let i=0; i<20; i++) {
			g_sysTime += g_timeStep;
			moveParticles(g_timeStep);
		}
	}

	function moveParticles(dt) {
		const nn=g_nParticles;

		for (let i=0; i<nn; i++) {
			const a = 0.5*dt/g_mas[i];
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_xx[i] += g_vx[i]*dt;
			g_yy[i] += g_vy[i]*dt;
		}
		calcForce();
		for (let i=0; i<nn; i++) {
			const a = 0.5*dt/g_mas[i]
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
		}
		g_kineticEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_kineticEnergy += 0.5*g_mas[i]*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
		}
	}

	function calcForce() {
		const nn=g_nParticles, s05 = 0.5*3.418e-10;
		g_potentialEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_ffx[i]=0;g_ffy[i]=0;
		}
		for (let i=0; i<nn-1; i++) {
			for (let k=1; k<g_reg[i][0]; k++) {
				const j = g_reg[i][k];
				const xij=g_xx[i]-g_xx[j], yij=g_yy[i]-g_yy[j];
				const rij = Math.sqrt(xij*xij+yij*yij);
				if (rij<g_rCutoff) {
					const f = force(rij,g_kind[i],g_kind[j]);
					const fxij = f*xij/rij;
					const fyij = f*yij/rij;
					g_ffx[i] += fxij;
					g_ffy[i] += fyij;
					g_ffx[j] -= fxij;
					g_ffy[j] -= fyij;
				}
			}
		}
		for (let i=0; i<nn; i++) {
			g_ffx[i] += boundaryForce(g_xx[i]+s05)+boundaryForce(g_xx[i]-g_xMax-s05);
			g_ffy[i] += boundaryForce(g_yy[i]+s05)+boundaryForce(g_yy[i]-g_yMax-s05);
		}
	}

	function force(r,kndi,kndj) { // forceTable - linear interporation
		const hh=g_hh;
		const ir = Math.floor(r/hh);
		const a = r - ir*hh;
		g_potentialEnergy += ((hh-a)*g_potentialTable[kndi][kndj][ir] + a*g_potentialTable[kndi][kndj][ir+1])/hh;
		return ((hh-a)*g_forceTable[kndi][kndj][ir] + a*g_forceTable[kndi][kndj][ir+1])/hh;
	}

	function boundaryForce(r) {
		const ri = (3.418e-10/r);
		const r6 = ri*ri*ri*ri*ri*ri;
		g_potentialEnergy += 4.0*0.5*1.711e-21*r6*(r6-1.0); // epsilonOfAr = 1.711e-21
		return (24.0*0.5*1.711e-21*r6*(2.0*r6-1.0)/r);
	}

	//--- registeration

	function registerNearParticles() {
		const nn=g_nParticles;
		const rCut = g_rCutoff+20.0*2000.0*g_timeStep;
		const rcut2 = rCut*rCut;
		for (let i=0; i<nn-1; i++) {
			let k = 1;
			for (let j=i+1; j<nn; j++) {
				const r2 = (g_xx[i]-g_xx[j])*(g_xx[i]-g_xx[j])+(g_yy[i]-g_yy[j])*(g_yy[i]-g_yy[j]);
				if (r2<rcut2) {
					g_reg[i][k] = j;
					k = k + 1;
				}
			}
			g_reg[i][0] = k;
		}
	}

	function maxNearParticles() {
		const nn=g_nParticles;
		let mx=0;
		for (let i=0; i<nn-1; i++) {
			if (mx<g_reg[i][0]) mx = g_reg[i][0];
		}
		return (mx-1);
	}


	// --------------------  utility  --------------------

	function systemTemperature() {
		const nn=g_nParticles;
		let ek=0.0;  //kinetic energy (J)
		for (let i=0; i<nn; i++) {
			ek = ek + 0.5*g_mas[i]*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
		}
		return ek/(nn*g_kB);
	}

	function ajustVelocity(temp) {
		const nn=g_nParticles, a = Math.sqrt(temp/systemTemperature());
		for (let i=0; i<nn; i++) {
			g_vx[i] = a*g_vx[i];
			g_vy[i] = a*g_vy[i];
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( theme, boxSizeInNM, contTemp )
		evolve:			timeEvolution,			// timeEvolution( tempMode, contTemp)

		setTemp:		ajustVelocity,			// ajustVelocity( temp )

		getSysParam:	function() { return [ g_molecKind1, g_molecKind2, g_nParticles, g_timeStep, g_xMax, g_yMax ]; },
		getNow:			function() { return [ g_sysTime, systemTemperature(), g_kineticEnergy, g_potentialEnergy ]; },
		//getMorseData:	function(kind,index) { return g_Morse[kind][index]; },
		getKindStr:		function(kind) { return g_strOf[kind]; },

		getBondLength:	function(i,j) { return ( g_rBond[g_kind[i]] + g_rBond[g_kind[j]] ); },
		getNearList:	function(i) { return g_reg[i]; },
		getAtomData:	function(i) { return [ g_kind[i], g_rCollision[g_kind[i]], g_rBond[g_kind[i]] ]; },
		getPosition:	function(i) { return [ g_xx[i], g_yy[i] ]; },
		getVelocity:	function(i) { return [ g_vx[i], g_vy[i] ]; },
		getForce:		function(i) { return [ g_ffx[i], g_ffy[i] ]; },
	};

})() // ====================  mixMMD2D end  ====================


const js017 = (function(){ // ====================  js Module  ====================

	const theModule = mixMMD2D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;
	let ctx;

	let v_theme = 0;  // 0:Fe-Al, 1:Ni-Pb, 2:Ni-Cr
	let v_BoxSizeInNM = 6.0;
	let v_contTemp = 300.0;
	let v_tempMode = 0;			// 0: adiabatic, 1: temp.control

	let p_kind1, p_kind2, p_nParticles, p_timeStep, p_xMax, p_yMax; // = theModule.getSysParam();

	let nCalc = 1;
	let dispMode = 0;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
  let breakFlag = false;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme, v_BoxSizeInNM, v_contTemp );
			[ p_kind1, p_kind2, p_nParticles, p_timeStep, p_xMax, p_yMax ] = theModule.getSysParam();
		}

		if ( !pauseFlag ) {
			for(let i=0; i<nCalc; i++ ) {
				theModule.evolve( v_tempMode, v_contTemp );
			}
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_tempMode, v_contTemp );
		}

		draw( ctx, dispMode );

		requestAnimationFrame(animate);
	}


	// --------------------  draw  --------------------

	const atomColor = [
		/*0:W */ "#6622ff", /*1:Mo*/ "#4444ff", /*2:Cr*/ "#0088ff", /*3:Fe*/ "#00ffff", /*4:Ni*/ "#00ff88",
		/*5:Al*/ "#00ff00", /*6:Pb*/ "#88ff00", /*7:Cu*/ "#cccc00", /*8:Ag*/ "#bbbb66", /*9:Ca*/ "#ddaa00" ];

	function draw( ctx, dispMode ) {
		const xp = 40, yp = 5, xSize = 400, ySize = 400, scale = xSize/p_xMax, yTextPos= 430;

		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		ctx.strokeStyle = "#888800";
		ctx.strokeRect( xp, yp, xSize, ySize );

		if ( dispMode==0 ) {
			drawBalls( ctx, scale, xp, yp );
		} else if ( dispMode==1 || dispMode==2 ) {
			strokeBalls( ctx, scale, xp, yp );
			drawBonds( ctx, dispMode-1, scale, xp, yp );
		}

		// caption
		let sysTime, temperature, kineticEnergy, potentialEnergy;
		[ sysTime, temperature, kineticEnergy, potentialEnergy ] = theModule.getNow();

		ctx.fillStyle = atomColor[p_kind1];
		ctx.fillText(theModule.getKindStr(p_kind1), xp, yTextPos+20);
		if ( p_kind1 != p_kind2 ) {
			ctx.fillStyle = atomColor[p_kind2];
			ctx.fillText(theModule.getKindStr(p_kind2), xp+30, yTextPos+20);
		}
		ctx.fillStyle = "#888888";
		ctx.fillText(`time = ${(sysTime*1.0e12).toFixed(1)} (ps)`, xp, yTextPos);
		ctx.fillText(`Temp = ${temperature.toFixed(1)} (K)`, xp+200, yTextPos);
		ctx.fillText(`,   N = ${p_nParticles}`, xp+60, yTextPos+20);
		ctx.fillText(`cont. Temp = ${v_contTemp.toFixed(0)} (K)`, xp+200, yTextPos+20);
		ctx.fillText(`Box = ${(p_xMax*1.0e9).toFixed(1)} x ${(p_yMax*1.0e9).toFixed(1)} (nm)`, xp, yTextPos+40);
		ctx.fillText(`Energy = ${(kineticEnergy+potentialEnergy).toExponential(4)} (J)`, xp+200, yTextPos+40);
		//document.getElementById("text_caption").innerHTML = "minimum molecular dynamics code"

	}

	function drawBalls( ctx, scale, xp, yp ) {
		const nn = p_nParticles, yMax = p_yMax, twoPi = 2.0*Math.PI;
		for (let i=0; i<nn; i++) {
			let x, y; [ x, y ] = theModule.getPosition(i);
			let kind, rCollision, rBond; [ kind, rCollision, rBond ] = theModule.getAtomData(i);
			ctx.fillStyle = atomColor[kind];
			ctx.beginPath();
			ctx.arc(x*scale+xp, (yMax-y)*scale+yp, rCollision*scale, 0, twoPi, false);
			ctx.fill();
		}
	}

	function strokeBalls( ctx, scale, xp, yp ) {
		const nn = p_nParticles, yMax = p_yMax, twoPi = 2.0*Math.PI;
		const d0 = theModule.getBondLength();
		for (let i=0; i<nn; i++) {
			let x, y; [ x, y ] = theModule.getPosition(i);
			let kind, rCollision, rBond; [ kind, rCollision, rBond ] = theModule.getAtomData(i);
			ctx.strokeStyle = "#444444";
			ctx.beginPath();
			ctx.arc(x*scale+xp, (yMax-y)*scale+yp, rBond*scale, 0, twoPi, false);
			ctx.stroke();
		}
	}

	function drawBonds( ctx, bondMode, scale, xp, yp ) { // single kind only - using nearList
		const nn = p_nParticles, yMax = p_yMax, pi = Math.PI;
		for (let i=0; i<nn-1; i++) {
			let xi, yi; [ xi, yi ] = theModule.getPosition(i);
			const nearList = theModule.getNearList(i);
			for (let k=1; k<nearList[0]; k++) {
				const j = nearList[k];
				let xj, yj; [ xj, yj ] = theModule.getPosition(j);
				const xij = xi-xj, yij = yi-yj;
				const dij = Math.sqrt(xij*xij+yij*yij);
				const d0 = theModule.getBondLength(i,j);
				if ( dij<1.2*d0 ) {
					let hue;
					if ( bondMode==0 ) { // bond color -- length
						hue = Math.floor((dij/d0-1.0)*900+120);
					} else if ( bondMode==1 ) { // bond color -- direction
						const th = 3.0*(Math.atan2(yij,xij)+0.5*pi)/pi;
						hue = Math.floor((th-Math.floor(th))*360.0);
					}
					if (hue<0) hue = 0;
					if (hue>270) hue = 270;
					const color = `hsl(${hue},100%,50%)`;
					drawLine(ctx, xi*scale+xp, (yMax-yi)*scale+yp, xj*scale+xp, (yMax-yj)*scale+yp, color );
				}
			}
		}
	}

	function drawLine(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	}


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}


	function setTempMode() {
		v_tempMode = 0 + document.getElementById("slct_tempMode").selectedIndex;
	}

	function setContTemp() {  // range mixing
		v_contTemp = Number(document.getElementById("range_temp").value);
		document.getElementById("text_temp").innerHTML = " " + v_contTemp.toFixed(0);
	}

	function setDispMode() {
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function setNcalc() {
		nCalc = 1 + document.getElementById("slct_nCalc").selectedIndex;
	}

	function breakLoop() {
		breakFlag = true;
	}


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		// dom control
		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setTempMode:	setTempMode,	// setTempMode()
		setContTemp:	setContTemp,	// setContTemp()
		setDispMode:	setDispMode,	// setDispMode()
		setNcalc:		setNcalc,		// setNcalc()
    breakLoop:		breakLoop,		// breakLoop();
	};

})(); // ====================  js017 module end  ====================


const js = js017;
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js017] mixture (Morse potential) MD2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" width="480" height="480"></canvas>
<br>

<label>theme: </label>
<select id="slct_theme" onChange="js.setTheme()">
<option>Fe-Al</option><option>Ni-Pb</option><option>Ni-Cr</option>
</select>
    <span style="margin-right: 200px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>Temp mode:</label>
<select id="slct_tempMode" onChange="js.setTempMode()">
<option selected>adiabatic</option><option>Temp.control</option>
</select>
<br>

<label>cont.Temp:</label><label id="text_temp"> 300</label>
<input type="range" id="range_temp" min="10" max="1000" value="300" step="5"
style="width:360px" oninput="js.setContTemp()">
<br>

<label> disp. mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option selected>ball</option><option>bond length</option><option>bond direction</option>
</select>
    <span style="margin-right: 80px;"></span>
<label>speed(nCalc/frame):</label>
<select id="slct_nCalc" onChange="js.setNcalc()">
<option selected>1</option><option>2</option><option>3</option><option>4</option>
<option>5</option><option>6</option>
</select>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
<br>

</body>
</html>
